In [1]:
!pip install bertopic[visualization] --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 47.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.0 MB/s eta 0:00:00


In [236]:
'''
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
BASE_DIR = "/content/drive/My Drive"
#!ls "/content/drive/My Drive"
!ls "/content/drive/My Drive/Colab Notebooks/nlp_bert_topic/"
'''
import os
BASE_DIR = os.getcwd()

DATA_DIR= EMBD_PATH = os.path.join(BASE_DIR, "nlp_bert_topic")

import pandas as pd
import os
import numpy as np
from copy import deepcopy

unique_region = ['Africa','Asia & Oceania','Europe','North America', 'Latin America & Caribbean']
LABEL_NAMES = [ 'Refuse','Rethink','Reduce','Reuse','Repair','Refurbish','Remanufacture','Repurpose', 'Recycle','Recover']
year_li = [2014,2015,2016,2017,2018]

In [237]:
org_df = pd.read_csv(os.path.join(DATA_DIR,"combined_output.csv"))
print(org_df.shape)
print(org_df.region.isnull().sum())
print(org_df.clean_subustype.isnull().sum())

org_df= org_df.dropna(subset=['clean_subustype'])
org_df.shape

(179452, 21)
0
99797


(79655, 21)

Remove stopwords

In [238]:
clean_dic = {"saloon" : 'salon', 
             'supple' : 'supply',
             'hairdresse' : 'hairdresser' , 
             'hairdresserr' : 'hairdresser' , 
             'agricultural': 'agriculture',
             'restuarant':'restaurant',
             'restaurante':'restaurant',
             'resturant' : 'restaurant',
             'restorant' : 'restaurant',
             'storera':'store',
             'good': 'goods',
             'shopkeepe':'shopkeeper',
             'motocicle':'motorcycle',
             'motocycle' : 'motorcycle',
             'moto':'motor',  
             'motorr' : 'motor',
             'cybercaf':'cybercafe',
             'cloth' : 'clothes',
             'onlie':'online',
             'soft drink': 'softdrink',
             'second hand' : 'secondhand',
             'fast food':'fastfood',
             'show room': 'showroom',
             'tomatoe':'tomato',
             'breed' :'breeding',
             'hand bag': 'handbag',
             'cyber cafe': 'cybercafe',
             'caf' : 'cafe',
             'internet caf':'cybercafe',
             'cafee' : 'cafe',
             'dry cleaning' :'laundry',
             'second class':'secondhand',
             'ready wear' : 'readywear',
             'clean' : 'cleaning',
             'sew': 'sewing',
             'vend': 'vendor',
             'architecture' : 'architectural',
             'consult' : 'consultancy',
             'clotheing' : 'clothing',
             'clothesing' : 'clothing',
             'clothee' : 'clothes',
             'vendoror' : 'vendor',
             'cell phone' : 'mobile phone',
             'cel phone' : 'mobile phone',
             'consultancyancy' : 'consultancy',
             'consultancying' : 'consultant',
             'grocery grocery': 'grocery',
             'health care' : 'healthcare',
             'hair stylish' : 'hairstylist',
             'sewinging' : 'sewing',
             'sewinger': 'sewing',
             'day care' :'daycare',
             'breedinging' : 'breeding',
             'clothese' : 'clothes'}

#https://stackoverflow.com/questions/51240862/find-and-replace-multiple-words-in-a-file-python
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [239]:
#https://towardsdatascience.com/text-normalization-with-spacy-and-nltk-1302ff430119

#spaCy Code Initialization:
import spacy

nlp = spacy.load('en_core_web_sm')

custom_words_to_remove= [' need', 'level', 'barranquilla', 'know', 
                         'basic','sell', 'sale', 'buy',"shop",  "business" , 
                          "customer", 'person', 'single', 'service',"consumer","store","typical",
                         "company", "general","private",'public',"resident","people","product","base"
                        "living", "area","want","like","room",'ld','ltd','american', 'item', 'answer', 'mean','michel',
                         'jose','de', 'etc', 'da','use','outer','inner','stall','don',
                        'non','predominate','middle','domain','detailed','activity','rican','base']
                        
def spacy_process(text):
    doc = nlp(text)
    
    #Tokenization and lemmatization are done with the spacy nlp pipeline commands
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
    #print("Tokenize+Lemmatize:")

    #Filter the stopword
    filtered_sentence =[] 
    for index, word in enumerate(lemma_list):
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False and len(word)>1 and  word not in custom_words_to_remove:
            filtered_sentence.append(word) 
            
    combined_word = " ".join(filtered_sentence)
    return replace_all(combined_word, clean_dic)

text = " we will show how to ducks good supple buy  sell laundry fishing stopwords using spacy i ltd tmp library know house basic time level housing soft drink weld"
print("Original text  : ",text)
print("Text after removing stopwords  : ",spacy_process(text))

Original text  :   we will show how to ducks good supple buy  sell laundry fishing stopwords using spacy i ltd tmp library know house basic time level housing soft drink weld
Text after removing stopwords  :  duck goods supply laundry fishing stopword spacy tmp library house time housing softdrink weld


In [240]:
org_df['clean_subustype_wo_stopwords'] = org_df['clean_subustype'].apply(spacy_process)
org_df['clean_subustype_wo_stopwords'].head(5)

org_df['clean_subustype_wo_stopwords'].replace('', np.nan, inplace=True)
org_df.dropna(subset=['clean_subustype_wo_stopwords'], inplace=True)
org_df
org_df.to_csv(os.path.join(DATA_DIR,"combined_df_wo_stowords.csv"))

In [246]:
q3_val = 0.9
org_df= pd.read_csv(os.path.join(DATA_DIR,"combined_df_wo_stowords.csv"))

if(q3_val == 0.75):
    Q3_fileName = 'table_Q3_75.csv'
    Valid_df_fileName = "Valid_df_75_wo_stowords.csv"

    topic_modelling_fileName= "Topic_modelling_output_75.csv"
    topic_modelling_formatted_fileName="Topic_modelling_output_formatted_75.csv"

    word_count_fileName= "word_count_output_75.csv"
    word_count_formatted_fileName="word_count_result_75.csv"

else:
    Q3_fileName = 'table_Q3_90.csv'
    Valid_df_fileName = "Valid_df_90_wo_stowords.csv"

    topic_modelling_fileName= "Topic_modelling_output_90_wo_Rs.csv"
    topic_modelling_formatted_fileName="Topic_modelling_output_formatted_90_wo_Rs.csv"

    word_count_fileName= "word_count_output_90_wo_Rs.csv"
    word_count_formatted_fileName="word_count_result_90_wo_Rs.csv"

In [242]:
def extract_valid(raw_df, df_q3_vals):
    new_df = pd.DataFrame(columns = raw_df.columns)
    insertion_index = 0

    for i in range(len(unique_region)):
        for j in range(len(year_li)):
            valid_q3 = df_q3_vals[(df_q3_vals['region']== unique_region[i] ) & ( df_q3_vals['yrsurv']== year_li[j])]
            raw_reg =  raw_df[(raw_df['region']== unique_region[i] ) & ( raw_df['yrsurv']== year_li[j]) ]

            for index, row in raw_reg.iterrows():
                if(row['weighted_sum'] >= valid_q3['weighted_sum'].values[0]):
                    new_df.loc[insertion_index] = row
                    insertion_index+=1
                    
                '''
                else:
                    #Filing mean of output
                    for k in range(len(LABEL_NAMES)):
                        if(row[LABEL_NAMES[k]] >= valid_q3[LABEL_NAMES[k]].values[0]):
                            new_df.loc[insertion_index] = row
                            insertion_index+=1
                            break
                '''
    return new_df

df_q3 = pd.read_csv(os.path.join(DATA_DIR, Q3_fileName))
valid_df = extract_valid(org_df, df_q3)  
valid_df.to_csv(os.path.join(DATA_DIR,Valid_df_fileName))  

##90 percentile

In [245]:
print(org_df.shape[0], valid_df.shape[0])
print(org_df.shape[0]-valid_df.shape[0])

valid_df = pd.read_csv(os.path.join(DATA_DIR,Valid_df_fileName))
df = valid_df.copy()
df.shape

78078 7670
70408


(7670, 24)

## Topic Modelling 

In [59]:
from sklearn.datasets import fetch_20newsgroups
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

sentence_model = SentenceTransformer("all-mpnet-base-v2")

In [213]:
#https://github.com/MaartenGr/BERTopic/issues/461
#https://maartengr.github.io/BERTopic/faq.html#how-can-i-run-bertopic-without-an-internet-connection

def get_topic_model(data,embeddings):
    random.seed(123)
    np.random.seed(123)
    torch.manual_seed(123)
        
    # preprocess data
    vectorizer_model = CountVectorizer(ngram_range=(1, 1), min_df=2)

    # do the BERT topic modelling
    umap_model = UMAP(n_neighbors=41, n_epochs= 500, n_components=3, min_dist=1e-3, metric='cosine', random_state=42)

    min_clusters = round(len(data) * 0.0020)
    print("min_clusters :: ",min_clusters)
    hdbscan_model = HDBSCAN(min_cluster_size= min_clusters, metric='euclidean', 
                            cluster_selection_method='eom', prediction_data=True, min_samples=5)

    #run the model
    topic_model = BERTopic(nr_topics = 35, umap_model=umap_model, hdbscan_model=hdbscan_model,
                         embedding_model = sentence_model, low_memory=True, calculate_probabilities=True)
    topics, probs = topic_model.fit_transform(data, embeddings)

    return topic_model

In [178]:
def create_embedding(region,data):
    EMBD_PATH = os.path.join(DATA_DIR, str(q3_val))
    if not os.path.exists(EMBD_PATH):
        os.makedirs(EMBD_PATH)

    file_name = region+".npy"
    file_path = os.path.join(EMBD_PATH, file_name)
    
    if(os.path.isfile(file_path)):
        all_embeddings = np.load(file_path)
    else:
        embeddings = sentence_model.encode(data)
        all_embeddings = np.array(embeddings)
        np.save(file_path, all_embeddings)
        all_embeddings = np.load(file_path)

    return all_embeddings

In [179]:
def print_model_result(model_list):
    new_df = pd.DataFrame()
    all_out = pd.DataFrame()
    for i, name in enumerate(unique_region):
        model = model_list[i]
        tmp = model.get_topic_freq()[0:6]
        topics = []
        topic_names =[]

        topic_out_df = pd.DataFrame()
        for j in range(6):
            selected_topic = model.get_topic(j)
            topics.append(selected_topic)

            names =[]
            for k in range(len(selected_topic)):
                names.append(selected_topic[k][0])
            topic_out_df[j] = names[0:7]
            topic_names.append(names)

        tmp['region'] = unique_region[i]
        tmp['topic_names'] = topic_names
        tmp['topics'] = topics

        topic_out_df['region'] =  unique_region[i]
        all_out = pd.concat([all_out, topic_out_df], ignore_index=True, sort=False)

        new_df = new_df.append(tmp)

    new_df.to_csv(os.path.join(DATA_DIR, topic_modelling_fileName))
    all_out.to_csv(os.path.join(DATA_DIR, topic_modelling_formatted_fileName))
    
    return all_out

In [247]:
from bertopic import BERTopic
import random
import numpy as np
import torch

unique_region = ['Africa','Asia & Oceania','Europe','North America', 'Latin America & Caribbean']
unique_region__ = ['North America','Latin America & Caribbean']

unique_region_ = ['Latin America & Caribbean']
def return_model():

    model_list = []
    for selected_region in unique_region:
        regional_df =  df[df['region']== selected_region]
        docs = list(regional_df.loc[:, "clean_subustype_wo_stopwords"].values)
        '''
        # Set the random seed for reproducibility
        random.seed(123)
        np.random.seed(123)
        torch.manual_seed(123)
        '''
        embeddings= create_embedding(selected_region,docs)
        model = get_topic_model(docs,embeddings)

        print("Selected _region",selected_region)
        print(model.get_topic_freq())
        model_list.append(model)
    
    return model_list


In [249]:
model_list3= return_model()
print("Result for 0.90 try1")
res3_90  = print_model_result(model_list3)
res3_90

min_clusters ::  2
Selected _region Africa
    Topic  Count
0      -1    220
1       0     67
2       1     64
3       2     63
4       3     60
5       4     50
6       5     49
7       6     33
8       7     29
9       8     28
10      9     27
11     10     25
12     11     24
13     12     21
14     13     20
15     14     18
18     15     14
17     16     14
16     17     14
19     18     13
20     19     13
21     20     12
22     21     12
23     22     10
24     23     10
26     24      9
25     25      9
27     26      8
28     27      7
29     28      6
30     29      6
31     30      5
32     31      5
33     32      3
34     33      2
min_clusters ::  4
Selected _region Asia & Oceania
    Topic  Count
0      -1    380
1       0    234
2       1    180
3       2    147
4       3    108
5       4     95
6       5     82
8       6     77
7       7     77
9       8     75
10      9     53
11     10     52
12     11     50
13     12     48
14     13     45
15     14     38
16   

,0,1,2,3,4,5,region
0,trade,clothing,beer,motorcycle,electronic,vegetable,Africa
1,tertiary,secondhand,softdrink,mechanic,appliance,leafy,Africa
2,egg,clothes,recreational,car,electrical,bean,Africa
3,car,goods,food,repair,hardware,doughnut,Africa
4,family,retail,drink,bicycle,refrigerator,grow,Africa
5,scooter,woman,restaurant,automobile,fix,corn,Africa
6,online,garment,vehicle,maintenance,computer,nut,Africa
7,woman,phone,car,client,fruit,contractor,Asia & Oceania
8,clothes,mobile,motorcycle,industry,vegetable,construction,Asia & Oceania
9,clothing,electronic,workshop,support,juice,building,Asia & Oceania


In [250]:
model_list5= return_model()
print("Result for 0.90 try2")
res5_90_wo_R = print_model_result(model_list5)
print(res5_90_wo_R.equals(res3_90))
res5_90_wo_R

min_clusters ::  2
Selected _region Africa
    Topic  Count
0      -1    220
1       0     67
2       1     64
3       2     63
4       3     60
5       4     50
6       5     49
7       6     33
8       7     29
9       8     28
10      9     27
11     10     25
12     11     24
13     12     21
14     13     20
15     14     18
18     15     14
17     16     14
16     17     14
19     18     13
20     19     13
21     20     12
22     21     12
23     22     10
24     23     10
26     24      9
25     25      9
27     26      8
28     27      7
29     28      6
30     29      6
31     30      5
32     31      5
33     32      3
34     33      2
min_clusters ::  4
Selected _region Asia & Oceania
    Topic  Count
0      -1    380
1       0    234
2       1    180
3       2    147
4       3    108
5       4     95
6       5     82
8       6     77
7       7     77
9       8     75
10      9     53
11     10     52
12     11     50
13     12     48
14     13     45
15     14     38
16   

,0,1,2,3,4,5,region
0,trade,clothing,beer,motorcycle,electronic,vegetable,Africa
1,tertiary,secondhand,softdrink,mechanic,appliance,leafy,Africa
2,egg,clothes,recreational,car,electrical,bean,Africa
3,car,goods,food,repair,hardware,doughnut,Africa
4,family,retail,drink,bicycle,refrigerator,grow,Africa
5,scooter,woman,restaurant,automobile,fix,corn,Africa
6,online,garment,vehicle,maintenance,computer,nut,Africa
7,woman,phone,car,client,fruit,contractor,Asia & Oceania
8,clothes,mobile,motorcycle,industry,vegetable,construction,Asia & Oceania
9,clothing,electronic,workshop,support,juice,building,Asia & Oceania
